In [1]:
import os
import pandas as pd
import s3fs

S3_BUCKET = os.getenv("S3_BUCKET", "s3-data-quest.rearc.io")
fs = s3fs.S3FileSystem(anon=True)

import json
with fs.open(f"{S3_BUCKET}/datausa.api.json") as f:
	usa_json=json.load(f)
	
usa_data=pd.DataFrame(usa_json['data'])
usa_data

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2019,2019,328239523,united-states
1,01000US,United States,2018,2018,327167439,united-states
2,01000US,United States,2017,2017,325719178,united-states
3,01000US,United States,2016,2016,323127515,united-states
4,01000US,United States,2015,2015,321418821,united-states
5,01000US,United States,2014,2014,318857056,united-states
6,01000US,United States,2013,2013,316128839,united-states


In [2]:
# 1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.
usa_select_years = usa_data[(2013 <= usa_data["ID Year"]) & (usa_data["ID Year"] <= 2018)]
print(usa_select_years)
mean = usa_select_years['Population'].mean()
std = usa_select_years['Population'].std()
print(f"Population Mean: {mean}")
print(f"Standard Deviation: {std}")

  ID Nation         Nation  ID Year  Year  Population    Slug Nation
1   01000US  United States     2018  2018   327167439  united-states
2   01000US  United States     2017  2017   325719178  united-states
3   01000US  United States     2016  2016   323127515  united-states
4   01000US  United States     2015  2015   321418821  united-states
5   01000US  United States     2014  2014   318857056  united-states
6   01000US  United States     2013  2013   316128839  united-states
Population Mean: 322069808.0
Standard Deviation: 4158441.040908095


In [3]:
# read in the pr gov data using first row as the header and the space-like characters (regex) as delimiters
pr_data = pd.read_csv(f"s3://{S3_BUCKET}/pr.data.0.Current", header=[0], delimiter=r"\s+")

# drop the unused footnote column
pr_data.drop(columns=["footnote_codes"], inplace=True)
pr_data

,series_id,year,period,value
0,PRS30006011,1995,Q01,2.6
1,PRS30006011,1995,Q02,2.1
2,PRS30006011,1995,Q03,0.9
3,PRS30006011,1995,Q04,0.1
4,PRS30006011,1995,Q05,1.4
...,...,...,...,...
4104,PRS30006162,2016,Q04,-1.0
4105,PRS30006162,2016,Q05,-0.1
4106,PRS30006162,2017,Q01,-3.2
4107,PRS30006162,2017,Q02,2.4


In [4]:
# 2. Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. 
pr_summed_by_year = pr_data.groupby(['series_id', 'year']).sum().reset_index()
pr_summed_by_year

,series_id,year,value
0,PRS30006011,1995,7.1
1,PRS30006011,1996,-0.5
2,PRS30006011,1997,4.4
3,PRS30006011,1998,4.2
4,PRS30006011,1999,-7.7
...,...,...,...
1074,PRS30006162,2013,7.5
1075,PRS30006162,2014,-3.4
1076,PRS30006162,2015,-12.0
1077,PRS30006162,2016,1.5


In [5]:
pr_summed_by_year[pr_summed_by_year['series_id'] == 'PRS30006012']

,series_id,year,value
27,PRS30006012,1995,1.8
28,PRS30006012,1996,0.6
29,PRS30006012,1997,6.8
30,PRS30006012,1998,-0.1
31,PRS30006012,1999,-8.0
32,PRS30006012,2000,-1.6
33,PRS30006012,2001,-35.4
34,PRS30006012,2002,-28.0
35,PRS30006012,2003,-22.3
36,PRS30006012,2004,-1.1


In [6]:
best_year_idx = pr_summed_by_year.drop(columns='year').groupby(['series_id']).idxmax().reset_index(drop=True)
pr_best_years = pr_summed_by_year.loc[best_year_idx.value]
pr_best_years

,series_id,year,value
17,PRS30006011,2012,9.500
46,PRS30006012,2014,8.800
57,PRS30006013,1998,733.290
96,PRS30006021,2010,14.200
123,PRS30006022,2010,11.200
158,PRS30006023,2018,502.487
179,PRS30006031,2012,11.900
205,PRS30006032,2011,11.300
219,PRS30006033,1998,730.889
245,PRS30006041,1997,34.700


In [7]:
pop_data=usa_data[["ID Year","Population"]].rename(columns={"ID Year": "year"})
pop_data

,year,Population
0,2019,328239523
1,2018,327167439
2,2017,325719178
3,2016,323127515
4,2015,321418821
5,2014,318857056
6,2013,316128839


In [8]:
record = pr_data[(pr_data["series_id"] == "PRS30006032") & (pr_data["period"] == "Q01")]
record

,series_id,year,period,value
924,PRS30006032,1995,Q01,0.1
929,PRS30006032,1996,Q01,-4.4
934,PRS30006032,1997,Q01,2.6
939,PRS30006032,1998,Q01,1.0
944,PRS30006032,1999,Q01,-4.2
949,PRS30006032,2000,Q01,0.1
954,PRS30006032,2001,Q01,-6.0
959,PRS30006032,2002,Q01,-7.0
964,PRS30006032,2003,Q01,-5.8
969,PRS30006032,2004,Q01,2.3


In [9]:
# 3. Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)
joined = pd.merge(left=record, right=pop_data, left_on='year', right_on='year')
joined

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,2.2,316128839
1,PRS30006032,2014,Q01,-1.6,318857056
2,PRS30006032,2015,Q01,-1.2,321418821
3,PRS30006032,2016,Q01,-0.6,323127515
4,PRS30006032,2017,Q01,1.5,325719178
5,PRS30006032,2018,Q01,1.9,327167439
6,PRS30006032,2019,Q01,-3.1,328239523
